In [2]:
# Roberto Lucas de Oliveira Conceição
# Lucas Fereira da Silva
# Pedro Paulo Gomes Araújo
# Heitor Tomazeli Da Silva
# Amanda Verdan de Souza
# Jonathan Magacho Demetrio

In [3]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np 
import time, math, random
from typing import Tuple

# import gym 
import gym

In [4]:
env = gym.make('CartPole-v1')

In [5]:
policy = lambda obs: 0

for _ in range(20):
    obs = env.reset()
    for _ in range(80):
        actions = policy(obs)
        obs, reward, done, info = env.step(actions) 
        env.render()
        time.sleep(0.03)

env.close()

C:\Users\rober\AppData\Roaming\Python\Python38\site-packages\gym\envs\classic_control\cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


KeyboardInterrupt: 

In [6]:
# obs = possição, velocidade, angulo, velocidade angular
# actions = 0 para esquerda, 1 para direita

In [7]:
?env.env

In [8]:
policy = lambda _,__,___, tip_velocity : int( tip_velocity > 0 )
#Função de política simples

In [9]:
#Usando Qlearning

In [10]:
n_bins = ( 6 , 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]
#tranforma de continuopara discreto
def discretizer( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[angle, pole_velocity]])[0]))

In [11]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape
#Inicializa a Q tabela de valores 

(6, 12, 2)

In [12]:
def policy( state : tuple ):
    return np.argmax(Q_table[state])
#Cria uma função que use a tabela Q para selecionar o valor Q mais alto

In [13]:
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value
#faz a atualização da Q tabela

In [14]:
def learning_rate(n : int , min_rate=0.02 ) -> float  :
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))
#taxa de aprendizado

In [15]:
def exploration_rate(n : int, min_rate= 0.2 ) -> float :
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))
#taxa de exploração

In [ ]:
n_episodes = 500 
for e in range(n_episodes):
    
    current_state, done = discretizer(*env.reset()), False
    
    while done==False:
        
        # ação da politica
        action = policy(current_state) # exploit
        
        # inserir aleatoriamente
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # incremento
        obs, reward, done, _ = env.step(action)
        new_state = discretizer(*obs)
        
        #update
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        #visualização
        env.render()
            

In [ ]:
# Projeto gym Carpole 